<span type="title">从加减到乘除</span> | <span type="update">2018-07-14</span> | <span type="version">2</span>

<span type="intro"><p type="card-text">本章主要讲解乘法器和除法器。每种部件都从其二进制表示（如何实现二进制的乘法和除法操作）讲起，接着详述了实际利用寄存器和加法、减法器在CPU上实际实现乘法和除法器的过程。最后，讲解了我们为了减少计算机开销对乘法和除法器所做出的优化，包括减小计算机硬件开支（寄存器个数和长度）以及优化时间消耗（比如乘法器的并行运算）。</p></span>

# 二进制乘法运算

定义被乘数 Multiplicand， 乘数 Multiplier， 乘积 Product。为了得到Product，十进制乘法运算十分繁琐，对于乘数的每一位，都需要和被乘数进行九九运算表的查询操作，并且要进行一次加法运算来处理进位。对于n位的乘数，需要n次加法和n次表查询。这样仅仅得到了n个中间结果，还要对这些中间结果进行相加，那么一共需要2n次的加法，假设每次加法损耗T，一共损耗2T的时间。对于行波进位器而言，其损耗有4n+2个逻辑门的时间。

对于较为简单数字的运算，比如1000×1001，可以得到一个结论，如果乘数的位数为0，则被乘数向左移动一位，如果乘数的位数为1，则写下来乘数，最后将所有中间结果相加即可。

对于二进制而言，我们需要做的就是，每次向右移动乘数，向左移动被乘数，然后按照上述逻辑得到一个中间值，每次得到的中间值用来累加，得到最终的结果。

EDVAC是冯诺依曼提出的首个采用二进制运算的计算机，因为二进制和晶体管状态有很好的对应，并且尤其在乘法运算中节省大量开销，因此就成了最佳的选择。但是需要注意，输入输出设备总要承担二进制和十进制的转换工作。

![](w4p1.png)

如上图所示的运算过程。

# 乘法器的实现

![](w4p2.png)

如上图所示，采用一个8bit的寄存器保存被乘数，使用一个4bit的寄存器保存乘数，使用一个8bit的全加器和8bit的寄存器来保留运算结果，所有的控制电路有control test来执行。这个模型可以用来模拟4bit的两个二进制数相乘。

首先，初始化状态下，被乘数位于8bit寄存器的低四位，高四位填充为0，乘数位于4bit寄存器的四位，结果填充为0。

对于每一次循环，都要首先由control test判定是否到达4次（乘数位数为4，因此要加4次）。之后乘数向右移动一位，被乘数向左移动一位，如果乘数为1，则被乘数和结果输入加法器得到结果保存到结果寄存器中，如果乘数为0，则被乘数继续向左移动，乘数向右移动，重复这个循环，直到4次，最后结果寄存器保存的就是最终结果。

![](w4p3.png)

总体流程如上所示。

但是，这个乘法器有一些小问题，比如，如果按照这样的顺序逻辑，那么需要损耗3N个周期，那么32位的乘法就需要100个时钟周期，对于一个3Ghz的CPU而言，运算一个乘法就需要(1/3)*100≈33ns时间。这显然是无法接受的。

对乘法器的改进如下：

![](w4p4.png)

这样的改动同时进行了移位和加法，因为当时钟上升沿到来的时候，加法器正在运行加法，而寄存器的移位也正在进行，因为被乘数寄存器的移位并且传输到加法器需要时间，在这个时间内，基本上时钟周期已经过去，所以即便其已经进入加法器，也不会对结果寄存器产生影响。

![](w4p5.png)

如上图所示是另外一个优化，其目的在于减少不必要的硬件资源。因为我们只需要4bit的被乘数，但是使用了8bit的寄存器，此外，对于乘数，每次只用到了1bit，因此可以将乘数寄存器和结果寄存器搞在一起，作为一个8bit的寄存器。

在开始的时候，对于结果寄存器中最后一位进行判断，如果为1，则被乘数寄存器和结果寄存器前4位相加，得到结果保存在结果寄存器前四位，同时乘数寄存器向右移动（也就是结果寄存器向右移动，因为这个1bit已经没用了），之后重复进行这个过程。

对于N位乘法器而言，使用N位的被乘数寄存器和2N位的结果寄存器即可完成运算，这有效的减少了硬件损耗。

# 除法的运算过程

定义除法的名词：除数 Divisor 被除数 Dividend 商 Quotient 余数 Remainder，那么 被除数=除数×商＋余数

对于二进制除法而言，如下流程所示：

![](w4p6.png)

首先，将被除数看作余数，接着，余数和除数相减，如果结果大于0，则商为1，否则商为0，余数向右移动1，商向左移动1，接着循环执行，知道商为1，余数由余数减去除数所得，这时候更新余数，商为1，最终循环次数结束，得到商和余数。

这里面，对于4bit的除数，需要用到8bit的寄存器，寄存器不存在的位置，填充为0。对于商，则4bit即可，被除数和余数而言，需要8bit的寄存器。


# 除法器的实现

对于一个4bit的除法器而言，其需要8bit的除数寄存器，支持向右移动，一个4bit的商寄存器，支持向左移动，一个8bit的ALU，一个8bit的余数寄存器，用来写入结果。

![](w4p7.png)

初始化情况如上图所示，除数首先填入除数寄存器的高四位，然后在每次循环中向右移动一位。商初始化为0，每次循环向左移动一位。余数寄存器初始化保存被除数数据。

首先执行减法运算，余数寄存器R-除数寄存器D，将结果保存在余数寄存器R中。当余数寄存器中的数值小于0（商不足以为1，不够除，判定余数寄存器最高位即可），则撤销这一操作，执行加法运算，将余数寄存器的值和除数寄存器的值相加保存在余数寄存器中，然后商向左移动1位，新的右位设置为0。如果数值大于0，则商向左移动后，新的右位设置为1。

除数寄存器向右移动1位，检查循环次数，如果不足以结束，则继续第二轮，执行减法运算，重复下去。

一个32位的除法器，除了商是32位的，其余——余数寄存器和除数寄存器，以及ALU，都需要是64位的。

![](w4p8.png)

以上就是32bit除法器的工作流程图。

除法器的优化比较困难，如下所示是一种优化，这种方案取消了商寄存器，将其保留在余数寄存器的下32bit，而上32bit则保存原来的余数，每次正常的运算都需要余数寄存器像左移动一位，以让出地方给商。但是，如果余数小于0，则需要余数寄存器向右移动，来执行加法操作，因为余数寄存器的移动，所以除数寄存器就不需要移动了，因此其可以用32bit寄存器即可实现，这样就节省了1个64bit寄存器，ALU也只需要32bit即可。

![](w4p9.png)